# Load and reading the data 

In [2]:
from bs4 import BeautifulSoup

def extract_text_from_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    paragraphs = soup.find_all('p')
    text_content = "\n".join([p.get_text() for p in paragraphs])
    return text_content

# File paths for the different laws
file_paths = {
    'gdpr': '/kaggle/input/chroma-db-italian/Italian_gdpr.html',
    'ai_act': '/kaggle/input/chroma-db-italian/italian_AI_act.html',
    'dma': '/kaggle/input/chroma-db-italian/italian_dma.html',
    'dsa': '/kaggle/input/chroma-db-italian/italian_dsa.html'
}

# Extract text for each law
texts = {law: extract_text_from_html(path) for law, path in file_paths.items()}

# Example usage to print the first 1000 characters of each law's text
for law, text in texts.items():
    print(f"First 1000 characters of {law.upper()}:\n{text[:1000]}\n")


First 1000 characters of GDPR:
4.5.2016   
IT
Gazzetta ufficiale dell'Unione europea
L 119/1
REGOLAMENTO (UE) 2016/679 DEL PARLAMENTO EUROPEO E DEL CONSIGLIO
del 27 aprile 2016
relativo alla protezione delle persone fisiche con riguardo al trattamento dei dati personali, nonché alla libera circolazione di tali dati e che abroga la direttiva 95/46/CE (regolamento generale sulla protezione dei dati)
(Testo rilevante ai fini del SEE)
IL PARLAMENTO EUROPEO E IL CONSIGLIO DELL'UNIONE EUROPEA,
visto il trattato sul funzionamento dell'Unione europea, in particolare l'articolo 16,
vista la proposta della Commissione europea,
previa trasmissione del progetto di atto legislativo ai parlamenti nazionali,
visto il parere del Comitato economico e sociale europeo (1),
visto il parere del Comitato delle regioni (2),
deliberando secondo la procedura legislativa ordinaria (3),
considerando quanto segue:
(1)
La protezione delle persone fisiche con riguardo al trattamento dei dati di carattere personale 

# install the chroma db and some libraries

In [3]:
!pip install -U langchain-community
!pip install sentence_transformers
!pip install chromadb
import chromadb
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer
from langchain.embeddings import HuggingFaceBgeEmbeddings
from bs4 import BeautifulSoup

nltk.download('punkt')

tokenizer = AutoTokenizer.from_pretrained('bigscience/bloomz')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

# make the chunks

In [4]:
def chunk_text_based_on_tokens(text, max_tokens=300):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(tokenizer.tokenize(sentence))
        if current_length + sentence_length <= max_tokens:
            current_chunk.append(sentence)
            current_length += sentence_length
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_length = sentence_length

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

def extract_sections_articles_chapters(soup):
    sections = []
    current_section = []
    for element in soup.find_all(['h1', 'h2', 'h3', 'p']):
        if element.name in ['h1', 'h2', 'h3']:
            if current_section:
                sections.append(" ".join(current_section))
                current_section = []
            current_section.append(element.get_text())
        else:
            current_section.append(element.get_text())
    if current_section:
        sections.append(" ".join(current_section))
    return sections

def load_and_process_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = extract_sections_articles_chapters(soup)
    all_chunks = []
    for section in sections:
        all_chunks.extend(chunk_text_based_on_tokens(section))
    return all_chunks

def embed_chunks(chunks, model_name):
    encode_kwargs = {'normalize_embeddings': True}
    model_norm = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs={'device': 'cuda'},
        encode_kwargs=encode_kwargs
    )
    embeddings = model_norm.embed_documents(chunks)
    return embeddings

def setup_chroma_collection(client, collection_name):
    try:
        client.delete_collection(name=collection_name)
    except Exception as e:
        print(f"Error deleting collection: {e}")
    try:
        collection = client.create_collection(name=collection_name)
        return collection
    except Exception as e:
        print(f"Error creating collection: {e}")
        return None

def process_and_store_embeddings(file_path, collection_name, model_name, model_norm):
    chunks = load_and_process_html(file_path)
    embeddings = embed_chunks(chunks, model_name)

    chroma_client = chromadb.Client()
    collection = setup_chroma_collection(chroma_client, collection_name)
    
    for i, embedding in enumerate(embeddings):
        collection.add(
            documents=[chunks[i]],
            ids=[f"id_{i}"],
            embeddings=[embedding]
        )
    return collection, chunks, embeddings

def embed_and_query(query, model_norm, collection, top_k=10):
    query_embedding = embed_query(query, model_norm)
    results = query_chroma_db(query_embedding, collection, top_k)
    return results

def embed_query(query, model_name):
    query_embedding = model_name.embed_documents([query])
    return query_embedding[0]

def query_chroma_db(query_embedding, collection, top_k=10):
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results

In [5]:
# File paths and collection names for the different laws
laws_info = {
    'gdpr': {
        'file_path': '/kaggle/input/chroma-db-italian/Italian_gdpr.html',
        'collection_name': 'embeddings_gdpr',
        'query': "In che modo il diritto degli Stati membri dovrebbe conciliare la libertà di espressione e di informazione con il diritto alla protezione dei dati personali, specialmente nel contesto del trattamento dei dati a fini giornalistici, accademici, artistici o letterari?"
    },
    'ai_act': {
        'file_path': '/kaggle/input/chroma-db-italian/italian_AI_act.html',
        'collection_name': 'embeddings_ai_act',
        'query': "Quali sono le principali raccomandazioni per i fornitori di sistemi di IA non ad alto rischio secondo il testo, e come queste potrebbero contribuire alla diffusione di un'IA etica e affidabile nell'Unione Europea?"
    },
    'dma': {
        'file_path': '/kaggle/input/chroma-db-italian/italian_dma.html',
        'collection_name': 'embeddings_dma',
        'query': "Quali sono i poteri delegati alla Commissione dall'articolo 290 del TFUE in merito alla modifica degli allegati del regolamento dell'Unione europea riguardante i prodotti a duplice uso, e quale ruolo giocano il Parlamento europeo e il Consiglio nella preparazione di tali atti delegati?"
    },
    'dsa': {
        'file_path': '/kaggle/input/chroma-db-italian/italian_dsa.html',
        'collection_name': 'embeddings_dsa',
        'query': "Quali sono le principali responsabilità della Commissione in caso di violazione delle disposizioni del regolamento da parte delle piattaforme online di dimensioni molto grandi o dei motori di ricerca online di dimensioni molto grandi, e quali azioni può intraprendere per garantire il rispetto del regolamento stesso?"
    }
}

model_name = "microsoft/deberta-v3-base"
encode_kwargs = {'normalize_embeddings': True}

model_norm = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

# Process and store embeddings for each law
collections = {}
chunks_dict = {}
embeddings_dict = {}

for law, info in laws_info.items():
    print(f"Processing {law}...")
    collection, chunks, embeddings = process_and_store_embeddings(info['file_path'], info['collection_name'], model_name, model_norm)
    collections[law] = collection
    chunks_dict[law] = chunks
    embeddings_dict[law] = embeddings

# Utility function to print specific chunk or embedding
def print_specific_chunk_or_embedding(law, index, data_type='chunk'):
    if data_type == 'chunk':
        if law in chunks_dict and len(chunks_dict[law]) > index:
            print(f"{index + 1}th chunk of {law.upper()}:\n{chunks_dict[law][index]}\n")
        else:
            print(f"{law.upper()} does not have {index + 1} chunks or law not found.")
    elif data_type == 'embedding':
        if law in embeddings_dict and len(embeddings_dict[law]) > index:
            print(f"{index + 1}th embedding of {law.upper()}:\n{embeddings_dict[law][index]}\n")
        else:
            print(f"{law.upper()} does not have {index + 1} embeddings or law not found.")
    else:
        print("Invalid data type specified. Use 'chunk' or 'embedding'.")


# Query each law and print results
for law, info in laws_info.items():
    print(f"\nQuerying {law.upper()} collection:")
    results = embed_and_query(info['query'], model_norm, collections[law], top_k=1)
    
    # Ensure only one result is retrieved
    if results['documents']:
        retrieved_context = results['documents'][0]
        chunk_id = results['ids'][0][0]  # Accessing the first element in the list of IDs

        # Print the chunk number and law name
        print(f"Retrieved chunk {chunk_id.split('_')[-1]} from {law.upper()}:")
        print(retrieved_context)
    else:
        print(f"No results found for {law.upper()}.")



config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Processing gdpr...
Error deleting collection: Collection embeddings_gdpr does not exist.
Processing ai_act...
Error deleting collection: Collection embeddings_ai_act does not exist.
Processing dma...
Error deleting collection: Collection embeddings_dma does not exist.
Processing dsa...
Error deleting collection: Collection embeddings_dsa does not exist.

Querying GDPR collection:
Retrieved chunk 124 from GDPR:
["(114) In ogni caso, se la Commissione non ha adottato alcuna decisione circa il livello adeguato di protezione dei dati di un paese terzo, il titolare del trattamento o il responsabile del trattamento dovrebbe ricorrere a soluzioni che diano all'interessato diritti effettivi e azionabili in relazione al trattamento dei suoi dati personali nell'Unione, dopo il trasferimento, così da continuare a beneficiare dei diritti fondamentali e delle garanzie. (115) Alcuni paesi terzi adottano leggi, regolamenti e altri atti normativi finalizzati a disciplinare direttamente le attività di 

In [6]:
print_specific_chunk_or_embedding('dsa', 130, 'chunk')

131th chunk of DSA:
A tale riguardo, ad esempio, il codice di condotta per lottare contro le forme illegali di incitamento all'odio online del 2016 stabilisce un parametro di riferimento per trattare le notifiche valide per la rimozione di forme illegali di incitamento all'odio in meno di 24 ore. I fornitori di piattaforme online di dimensioni molto grandi, in particolare quelli utilizzati principalmente per la diffusione al pubblico di contenuti pornografici, dovrebbero adempiere diligentemente a tutti i loro obblighi ai sensi del presente regolamento in relazione ai contenuti illegali che costituiscono violenza online, compresi i contenuti pornografici illegali, in particolare per quanto riguarda la garanzia che le vittime possano esercitare efficacemente i loro diritti in relazione ai contenuti che rappresentano la condivisione non consensuale di materiale intimo o manipolato attraverso il trattamento rapido delle segnalazioni e la rimozione di tali contenuti senza indebito ritardo.

In [7]:
model_norm = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs=encode_kwargs
)


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np

# Load BERT model and tokenizer for cosine similarity
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
model = AutoModel.from_pretrained("microsoft/deberta-v3-base")

# Load SentenceTransformer model for semantic similarity
semantic_model = SentenceTransformer('microsoft/deberta-v3-base')

def generate_bert_embedding(text, tokenizer, model, max_length=512):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=max_length)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].numpy()  # [CLS] token embedding
    return embedding


# Function to calculate cosine similarity
def calculate_cosine_similarity(reference_embedding, retrieved_embedding):
    return cosine_similarity(reference_embedding.reshape(1, -1), retrieved_embedding.reshape(1, -1))[0][0]

# Function to calculate semantic similarity using Sentence-Transformers
def calculate_semantic_similarity(reference_text, retrieved_text, model):
    embeddings1 = model.encode(reference_text, convert_to_tensor=True)
    embeddings2 = model.encode(retrieved_text, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(embeddings1, embeddings2)
    return similarity.item()

reference_answers = {
    'gdpr': "Il diritto degli Stati membri dovrebbe conciliare le norme che disciplinano la libertà di espressione e di informazione, comprese l'espressione giornalistica, accademica, artistica o letteraria, con il diritto alla protezione dei dati personali ai sensi del presente regolamento. Il trattamento dei dati personali effettuato unicamente a scopi giornalistici o di espressione accademica, artistica o letteraria dovrebbe essere soggetto a deroghe o esenzioni rispetto ad alcune disposizioni del presente regolamento se necessario per conciliare il diritto alla protezione dei dati personali e il diritto alla libertà d'espressione e di informazione sancito nell'articolo 11 della Carta. Ciò dovrebbe applicarsi in particolare al trattamento dei dati personali nel settore audiovisivo, negli archivi stampa e nelle emeroteche. È pertanto opportuno che gli Stati adottino misure legislative che prevedano le deroghe e le esenzioni necessarie ai fini di un equilibrio tra tali diritti fondamentali. Gli Stati membri dovrebbero adottare tali esenzioni e deroghe con riferimento alle disposizioni riguardanti i principi generali, i diritti dell'interessato, il titolare del trattamento e il responsabile del trattamento, il trasferimento di dati personali verso paesi terzi o a organizzazioni internazionali, le autorità di controllo indipendenti, la cooperazione e la coerenza nonché situazioni di trattamento dei dati specifiche. Qualora tali esenzioni o deroghe differiscano da uno Stato membro all'altro, dovrebbe applicarsi il diritto dello Stato membro cui è soggetto il titolare del trattamento. Per tenere conto dell'importanza del diritto alla libertà di espressione in tutte le società democratiche è necessario interpretare in modo esteso i concetti relativi a detta libertà, quali la nozione di giornalismo.",
    'ai_act': "Lo sviluppo di sistemi di IA diversi dai sistemi di IA ad alto rischio in conformità dei requisiti del presente regolamento può portare a una più ampia adozione nell'Unione di un'IA etica e affidabile. I fornitori di sistemi di IA non ad alto rischio dovrebbero essere incoraggiati a creare codici di condotta, che includano meccanismi di governance connessi, volti a promuovere l'applicazione volontaria di alcuni o tutti i requisiti obbligatori applicabili ai sistemi di IA ad alto rischio, adattati in funzione della finalità prevista dei sistemi e del minor rischio connesso e tenendo conto delle soluzioni tecniche disponibili e delle migliori pratiche del settore, come modelli e schede dati. I fornitori e, se del caso, i deployer di tutti i sistemi di IA, ad alto rischio o meno, e modelli di IA dovrebbero inoltre essere incoraggiati ad applicare su base volontaria requisiti supplementari relativi, ad esempio, agli elementi degli orientamenti etici dell'Unione per un'IA affidabile, alla sostenibilità ambientale, alle misure di alfabetizzazione in materia di IA, alla progettazione e allo sviluppo inclusivi e diversificati dei sistemi di IA, anche prestando attenzione alle persone vulnerabili e all'accessibilità per le persone con disabilità, la partecipazione dei portatori di interessi, con il coinvolgimento, se del caso, dei portatori di interessi pertinenti quali le organizzazioni imprenditoriali e della società civile, il mondo accademico, le organizzazioni di ricerca, i sindacati e le organizzazioni per la tutela dei consumatori nella progettazione e nello sviluppo dei sistemi di IA, e alla diversità dei gruppi di sviluppo, compreso l'equilibrio di genere. Per essere efficaci, i codici di condotta volontari dovrebbero basarsi su obiettivi chiari e indicatori chiave di prestazione che consentano di misurare il conseguimento di tali obiettivi. Essi dovrebbero inoltre essere elaborati in modo inclusivo, se del caso, con il coinvolgimento dei portatori di interessi pertinenti, quali le organizzazioni imprenditoriali e della società civile, il mondo accademico, le organizzazioni di ricerca, i sindacati e le organizzazioni per la tutela dei consumatori. La Commissione può elaborare iniziative, anche di natura settoriale, per agevolare la riduzione degli ostacoli tecnici che ostruiscono lo scambio transfrontaliero di dati per lo sviluppo dell'IA, anche per quanto riguarda l'infrastruttura di accesso ai dati e l'interoperabilità semantica e tecnica dei diversi tipi di dati.",
    'dma': "Al fine di consentire all’Unione di adeguarsi prontamente al mutare delle circostanze concernenti la valutazione della sensibilità delle esportazioni nel quadro delle autorizzazioni generali di esportazione dell’Unione oltre che agli sviluppi tecnologici e commerciali, è opportuno delegare alla Commissione il potere di adottare atti conformemente all’articolo 290 del trattato sul funzionamento dell’Unione europea (TFUE) riguardo alla modifica degli allegati I, II e IV del presente regolamento. Le decisioni relative all’aggiornamento dell’elenco comune di prodotti a duplice uso soggetti ai controlli sulle esportazioni di cui all’allegato I dovrebbero essere conformi agli obblighi e agli impegni che gli Stati membri o l’Unione hanno assunto in quanto membri dei pertinenti accordi internazionali di non proliferazione e in quanto membri dei regimi multilaterali in materia di controllo delle esportazioni oppure a seguito della ratifica dei pertinenti trattati internazionali. Nel caso in cui la modifica dell’allegato I riguardi prodotti a duplice uso elencati anche nell’allegato II o IV, tali allegati dovrebbero essere modificati di conseguenza. Le decisioni relative all’aggiornamento degli elenchi comuni di prodotti e destinazioni di cui alle sezioni da A ad H dell’allegato II dovrebbero essere adottate in considerazione dei criteri di valutazione di cui al presente regolamento. È di particolare importanza che durante i lavori preparatori la Commissione svolga adeguate consultazioni, anche a livello di esperti, nel rispetto dei principi stabiliti nell’accordo interistituzionale «Legiferare meglio» del 13 aprile 2016 (5). In particolare, al fine di garantire la parità di partecipazione alla preparazione degli atti delegati, il Parlamento europeo e il Consiglio ricevono tutti i documenti contemporaneamente agli esperti degli Stati membri, e i loro esperti hanno sistematicamente accesso alle riunioni dei gruppi di esperti della Commissione incaricati della preparazione di tali atti delegati.",
    'dsa': "Considerati i potenziali effetti significativi per la società di una violazione degli obblighi supplementari di gestione dei rischi sistemici che si applicano esclusivamente alle piattaforme online di dimensioni molto grandi e ai motori di ricerca online di dimensioni molto grandi, e al fine di rispondere a tali preoccupazioni di interesse pubblico, è necessario prevedere un sistema di vigilanza rafforzata delle azioni intraprese per far cessare o porre rimedio in modo effettivo alle violazioni del presente regolamento. Pertanto, una volta accertata e, ove necessario, sanzionata una violazione di una delle disposizioni del presente regolamento che si applicano esclusivamente alle piattaforme online di dimensioni molto grandi o ai motori di ricerca online di dimensioni molto grandi, la Commissione dovrebbe chiedere al fornitore di tale piattaforma o di tale motore di ricerca di elaborare un piano d'azione dettagliato per porre rimedio agli effetti della violazione per il futuro e di comunicare tale piano d'azione ai coordinatori dei servizi digitali, alla Commissione e al comitato entro un termine fissato dalla Commissione. La Commissione, tenendo conto del parere del comitato, dovrebbe stabilire se le misure incluse nel piano d'azione sono sufficienti per rispondere alla violazione, prendendo anche in considerazione l'inclusione o meno, tra le misure proposte, del rispetto del pertinente codice di condotta. La Commissione dovrebbe inoltre monitorare le misure successive adottate dal fornitore di una piattaforma online di dimensioni molto grandi o di un motore di ricerca online di dimensioni molto grandi in questione stabilite nel suo piano d'azione, tenendo conto anche di una revisione indipendente del fornitore. Se, a seguito dell'attuazione del piano d'azione, ritiene che non sia stato ancora posto interamente rimedio alla violazione, o se il piano d'azione non è stato fornito o non è ritenuto adeguato, la Commissione dovrebbe poter esercitare tutti i poteri di indagine o di esecuzione a norma del presente regolamento, compresi il potere di imporre penalità di mora e l'avvio della procedura per disabilitare l'accesso al servizio che ha commesso la violazione."
}

# Calculate and print similarities
similarities = []

for law, info in laws_info.items():
    print(f"\nQuerying {law.upper()} collection:")
    results = embed_and_query(info['query'], model_norm, collections[law], top_k=1)

    if results and 'documents' in results and results['documents']:
        retrieved_text = results['documents'][0][0]  # Assuming it's a list of lists
        
        # Generate embeddings using BERT for cosine similarity
        retrieved_embedding = generate_bert_embedding(retrieved_text, tokenizer, model)
        reference_embedding = generate_bert_embedding(reference_answers[law], tokenizer, model)

        # Calculate cosine similarity using BERT embeddings
        cosine_sim = calculate_cosine_similarity(reference_embedding, retrieved_embedding)

         # Calculate semantic similarity using Sentence-Transformers model
        semantic_sim = calculate_semantic_similarity(reference_answers[law], retrieved_text, semantic_model)

        # Store the results
        similarities.append({
            'law': law,
            'retrieved_answer': retrieved_text,
            'cosine_similarity': cosine_sim,
            'semantic_similarity': semantic_sim
        })

        # Print the results for this law
        chunk_id = results['ids'][0][0]  # Accessing the first element in the list of IDs
        print(f"Retrieved chunk {chunk_id.split('_')[-1]} from {law.upper()}:")
        print(retrieved_text)
        print(f"Cosine Similarity with reference answer: {cosine_sim:.4f}")
        print(f"Semantic Similarity with reference answer: {semantic_sim:.4f}")
        print("----\n")
    else:
        print(f"No valid results found for {law.upper()} in the query.")


Querying GDPR collection:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved chunk 124 from GDPR:
(114) In ogni caso, se la Commissione non ha adottato alcuna decisione circa il livello adeguato di protezione dei dati di un paese terzo, il titolare del trattamento o il responsabile del trattamento dovrebbe ricorrere a soluzioni che diano all'interessato diritti effettivi e azionabili in relazione al trattamento dei suoi dati personali nell'Unione, dopo il trasferimento, così da continuare a beneficiare dei diritti fondamentali e delle garanzie. (115) Alcuni paesi terzi adottano leggi, regolamenti e altri atti normativi finalizzati a disciplinare direttamente le attività di trattamento di persone fisiche e giuridiche poste sotto la giurisdizione degli Stati membri.
Cosine Similarity with reference answer: 0.9946
Semantic Similarity with reference answer: 0.9822
----


Querying AI_ACT collection:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved chunk 7 from DMA:
Tale coordinamento dovrebbe essere seguito da iniziative volte a introdurre controlli equivalenti a livello multilaterale al fine di ampliare la risposta al rischio individuato. (11) Anche la trasmissione di software e di tecnologie a duplice uso mediante mezzi elettronici, fax o telefono verso destinazioni al di fuori del territorio doganale dell’Unione dovrebbe essere sottoposta a controllo. Al fine di limitare gli oneri amministrativi a carico degli esportatori e delle autorità competenti degli Stati membri, è opportuno prevedere licenze generali o globali o interpretazioni armonizzate delle disposizioni per talune trasmissioni, come le trasmissioni a un cloud. (12) Alla luce dell’importante ruolo delle autorità doganali nell’applicazione dei controlli delle esportazioni, i termini utilizzati nel presente regolamento dovrebbero essere coerenti, nella misura del possibile, con le definizioni di cui al regolamento (UE) n. 952/2013 del Parlamento europeo e d

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved chunk 228 from DSA:
(154) In considerazione della portata e degli effetti dei rischi per la società che potrebbero essere causati dalle piattaforme online di dimensioni molto grandi e dai motori di ricerca online di dimensioni molto grandi, della necessità di affrontare tali rischi in via prioritaria e della loro capacità di prendere le misure necessarie, risulta giustificato limitare il periodo dopo il quale il presente regolamento inizia ad applicarsi ai fornitori di tali servizi.
Cosine Similarity with reference answer: 0.9826
Semantic Similarity with reference answer: 0.9689
----



In [9]:
model_norm = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},  # Using CUDA for faster computation
    encode_kwargs={'normalize_embeddings': True}
)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
integrated_questions_answers = [
    # Question 1 from GDPR
{
    'law': 'gdpr',
    'question': "Qual è il diritto fondamentale relativo al trattamento dei dati personali secondo la Carta dei diritti fondamentali dell'Unione Europea?",
    'answer': "La protezione delle persone fisiche in relazione al trattamento dei dati personali è un diritto fondamentale. L'articolo 8(1) della Carta dei diritti fondamentali dell'Unione Europea ('la Carta') e l'articolo 16(1) del Trattato sul funzionamento dell'Unione Europea (TFUE) stabiliscono che ogni persona ha il diritto alla protezione dei dati personali che la riguardano. Questo Regolamento mira a contribuire alla realizzazione di uno spazio di libertà, sicurezza e giustizia e di un'unione economica, al progresso economico e sociale, al rafforzamento e alla convergenza delle economie all'interno del mercato interno, e al benessere delle persone fisiche."
},
# Question 1 from AI Act
{
    'law': 'ai_act',
    'question': "Quali sono gli obiettivi principali dell'AI Act riguardo allo sviluppo e all'uso dell'AI nell'Unione Europea?",
    'answer': "L'AI Act mira a garantire che i sistemi di intelligenza artificiale immessi sul mercato e utilizzati nell'Unione siano sicuri, rispettino la legislazione esistente sui diritti fondamentali e i valori dell'Unione, e non compromettano i diritti fondamentali. L'atto mira a stabilire un quadro giuridico che affronti i rischi posti dall'AI, in particolare i sistemi di AI ad alto rischio, e mira a migliorare la trasparenza, la responsabilità e la fiducia nell'AI, promuovendo al contempo l'innovazione e la competitività."
},
# Question 1 from DMA
{
    'law': 'dma',
    'question': "Quali criteri vengono utilizzati per definire un 'gatekeeper' ai sensi del Digital Markets Act?",
    'answer': "Un gatekeeper ai sensi del DMA è definito come un fornitore di servizi di piattaforma di base che ha un impatto significativo sul mercato interno, funge da importante porta d'accesso per gli utenti commerciali per raggiungere gli utenti finali, e gode di una posizione radicata e durevole nel mercato. I criteri includono avere una forte posizione economica, un gran numero di utenti, e il controllo su un ecosistema che è difficile da contestare per altre aziende."
},
# Question 1 from DSA
{
    'law': 'dsa',
    'question': "Quali sono le principali responsabilità delle piattaforme online ai sensi del Digital Services Act?",
    'answer': "Ai sensi del DSA, le piattaforme online sono responsabili di adottare misure efficaci per mitigare i rischi legati ai contenuti illegali, garantire la sicurezza degli utenti e proteggere i diritti fondamentali. Le piattaforme devono implementare meccanismi per segnalare e rimuovere contenuti illegali, fornire agli utenti termini e condizioni chiari, e stabilire processi per la gestione dei reclami e degli appelli. Le piattaforme che raggiungono un numero significativo di utenti sono inoltre tenute a valutare e mitigare i rischi sistemici, come la diffusione di disinformazione e contenuti dannosi."
},
# Question 2 from GDPR
{
    'law': 'gdpr',
    'question': "Quali sono i principi fondamentali del trattamento dei dati personali ai sensi del GDPR?",
    'answer': "I principi fondamentali del trattamento dei dati personali ai sensi del GDPR includono la liceità, correttezza e trasparenza; limitazione delle finalità; minimizzazione dei dati; esattezza; limitazione della conservazione; integrità e riservatezza; e responsabilizzazione del titolare del trattamento."
},
# Question 2 from AI Act
{
    'law': 'ai_act',
    'question': "Cosa prevede l'AI Act riguardo alla classificazione dei sistemi di AI ad alto rischio?",
    'answer': "L'AI Act stabilisce una classificazione dei sistemi di AI ad alto rischio, che include sistemi che presentano un rischio elevato per la sicurezza, la salute, o i diritti fondamentali delle persone. Questi sistemi devono soddisfare requisiti più rigorosi per l'accuratezza, la robustezza e la trasparenza, e sono soggetti a obblighi di conformità prima di poter essere immessi sul mercato o utilizzati nell'Unione Europea."
},
# Question 2 from DMA
{
    'law': 'dma',
    'question': "Quali obblighi devono rispettare i 'gatekeeper' secondo il Digital Markets Act?",
    'answer': "I gatekeeper ai sensi del DMA devono rispettare vari obblighi che includono non combinare dati personali provenienti da servizi diversi senza il consenso degli utenti, consentire alle aziende di accedere ai dati generati dalle loro attività sulla piattaforma, e garantire la possibilità di disinstallare applicazioni preinstallate e utilizzare servizi alternativi."
},
# Question 2 from DSA
{
    'law': 'dsa',
    'question': "Come devono agire le piattaforme online per garantire la trasparenza nella pubblicità ai sensi del DSA?",
    'answer': "Le piattaforme online devono fornire informazioni chiare agli utenti su ogni annuncio pubblicitario visualizzato, compreso chi ha pagato per l'annuncio e perché l'utente specifico è stato preso di mira. Devono inoltre fornire informazioni su come funziona il sistema di raccomandazione utilizzato per indirizzare la pubblicità e offrire opzioni per modificarlo."
},
# Question 3 from GDPR
{
    'law': 'gdpr',
    'question': "Cosa significa 'diritto all'oblio' secondo il GDPR?",
    'answer': "Il 'diritto all'oblio' consente agli interessati di richiedere la cancellazione dei loro dati personali quando non sono più necessari per le finalità per cui sono stati raccolti, quando l'interessato revoca il consenso o quando i dati sono stati trattati illecitamente. Questo diritto è disciplinato dall'articolo 17 del GDPR."
},
# Question 3 from AI Act
{
    'law': 'ai_act',
    'question': "Quali misure devono adottare i fornitori di AI per garantire la conformità con il quadro normativo?",
    'answer': "I fornitori di AI devono effettuare valutazioni di conformità, stabilire sistemi di gestione dei rischi, garantire la trasparenza delle informazioni fornite agli utenti, e assicurare la qualità dei dati utilizzati per addestrare i modelli di AI. Devono inoltre mettere in atto meccanismi per monitorare e aggiornare i sistemi di AI."
},
# Question 3 from DMA
{
    'law': 'dma',
    'question': "Come può l'Unione Europea garantire il rispetto delle regole del Digital Markets Act da parte dei gatekeeper?",
    'answer': "L'Unione Europea può imporre sanzioni significative e misure correttive ai gatekeeper che non rispettano le regole del DMA, inclusa la possibilità di multe che possono raggiungere il 10% del fatturato globale annuo della società, nonché sanzioni periodiche di penalità. In casi estremi, possono essere presi provvedimenti strutturali."
},
# Question 3 from DSA
{
    'law': 'dsa',
    'question': "Che obblighi hanno le piattaforme online per quanto riguarda la moderazione dei contenuti?",
    'answer': "Le piattaforme online devono fornire meccanismi di segnalazione facilmente accessibili per consentire agli utenti di segnalare contenuti illegali e avere processi per esaminare queste segnalazioni in modo tempestivo. Devono inoltre fornire informazioni chiare e dettagliate agli utenti su eventuali decisioni di rimozione dei contenuti e consentire ricorsi contro tali decisioni."
},
# Question 4 from GDPR
{
    'law': 'gdpr',
    'question': "Cosa stabilisce il GDPR riguardo al trasferimento di dati personali verso paesi terzi?",
    'answer': "Il GDPR stabilisce che il trasferimento di dati personali verso paesi terzi o organizzazioni internazionali può avvenire solo se il paese terzo o l'organizzazione internazionale in questione garantisce un livello di protezione adeguato o se sono in atto garanzie adeguate, come le clausole contrattuali standard o il consenso esplicito dell'interessato."
},
# Question 4 from AI Act
{
    'law': 'ai_act',
    'question': "Quali sono i requisiti di trasparenza per i sistemi di AI secondo l'AI Act?",
    'answer': "I sistemi di AI devono essere progettati in modo da consentire la trasparenza delle loro operazioni. Devono includere informazioni chiare sull'uso di dati di addestramento, le capacità e le limitazioni dei sistemi, e devono garantire che gli utenti siano consapevoli del fatto che stanno interagendo con un sistema di AI, a meno che ciò non sia ovvio per il contesto."
},
# Question 4 from DMA
{
    'law': 'dma',
    'question': "Che ruolo svolge la Commissione Europea nel monitoraggio dei gatekeeper secondo il DMA?",
    'answer': "La Commissione Europea è responsabile dell'esecuzione e del monitoraggio del rispetto delle regole del DMA da parte dei gatekeeper. Ha il potere di avviare indagini di mercato, imporre sanzioni e monitorare il rispetto continuo degli obblighi da parte dei gatekeeper."
},
# Question 4 from DSA
{
    'law': 'dsa',
    'question': "Come si applicano le regole del DSA ai fornitori di hosting e ai motori di ricerca online?",
    'answer': "I fornitori di hosting e i motori di ricerca online devono rispettare gli obblighi del DSA che includono la rimozione rapida dei contenuti illegali, la fornitura di meccanismi di ricorso e un processo trasparente per le decisioni di moderazione dei contenuti. Devono anche assicurarsi che le informazioni degli utenti siano protette e gestire adeguatamente i reclami."
},
# Question 5 from GDPR
{
    'law': 'gdpr',
    'question': "Quali sono le sanzioni per la non conformità al GDPR?",
    'answer': "Le sanzioni per la non conformità al GDPR possono includere multe amministrative che possono arrivare fino a 20 milioni di euro o, nel caso di un'azienda, fino al 4% del fatturato globale annuo, a seconda di quale sia maggiore. Le sanzioni dipendono dalla gravità della violazione."
},
# Question 5 from AI Act
{
    'law': 'ai_act',
    'question': "Come affronta l'AI Act il rischio di bias nei sistemi di intelligenza artificiale?",
    'answer': "L'AI Act richiede che i sistemi di AI ad alto rischio siano sviluppati utilizzando dati di addestramento che sono rilevanti, rappresentativi, e privi di bias. Inoltre, i fornitori devono adottare misure per minimizzare i pregiudizi sistemici e garantire che i risultati dei sistemi siano equi e non discriminatori."
},
# Question 5 from DMA
{
    'law': 'dma',
    'question': "Quali azioni può intraprendere un gatekeeper per dimostrare il rispetto del DMA?",
    'answer': "Un gatekeeper può dimostrare il rispetto del DMA documentando le sue politiche aziendali, i cambiamenti apportati per rispettare gli obblighi, e fornendo prove di conformità alle regole stabilite dalla Commissione Europea. Può anche presentare rapporti periodici e collaborare con le autorità per il monitoraggio."
},
# Question 5 from DSA
{
    'law': 'dsa',
    'question': "Quali sono le misure di gestione del rischio che le piattaforme di grandi dimensioni devono adottare secondo il DSA?",
    'answer': "Le piattaforme di grandi dimensioni devono effettuare valutazioni annuali del rischio per identificare i rischi sistemici associati ai loro servizi, come la diffusione di contenuti illegali, la manipolazione intenzionale dei loro servizi o impatti negativi sui diritti fondamentali. Devono poi adottare misure per mitigare questi rischi."
},
# Question 6 from GDPR
{
    'law': 'gdpr',
    'question': "Quali sono i diritti degli interessati riguardo alla portabilità dei dati personali?",
    'answer': "Il diritto alla portabilità dei dati permette agli interessati di ricevere i propri dati personali che hanno fornito a un titolare del trattamento in un formato strutturato, di uso comune e leggibile da dispositivo automatico. Inoltre, consente il trasferimento diretto di questi dati a un altro titolare del trattamento, quando tecnicamente fattibile."
},
# Question 6 from AI Act
{
    'law': 'ai_act',
    'question': "Qual è il ruolo del comitato europeo per l'intelligenza artificiale proposto dall'AI Act?",
    'answer': "Il Comitato europeo per l'intelligenza artificiale è responsabile di supportare la Commissione Europea nel monitorare l'attuazione dell'AI Act, garantire la coerenza delle pratiche di applicazione tra gli Stati membri e facilitare la cooperazione tra le autorità nazionali di sorveglianza."
},
# Question 6 from DMA
{
    'law': 'dma',
    'question': "In che modo il DMA garantisce che i gatekeeper non favoriscano i propri prodotti o servizi?",
    'answer': "Il DMA vieta ai gatekeeper di applicare pratiche di preferenza ai propri prodotti o servizi rispetto a quelli di terzi. Questo include restrizioni sul preinstallare le proprie applicazioni e limitazioni sulla promozione esclusiva dei propri prodotti sulle piattaforme che gestiscono."
},
# Question 6 from DSA
{
    'law': 'dsa',
    'question': "Quali requisiti devono rispettare le piattaforme per il tracciamento pubblicitario ai sensi del DSA?",
    'answer': "Le piattaforme devono garantire che gli utenti possano facilmente identificare la pubblicità basata sul tracciamento dei dati e devono fornire opzioni chiare per disattivare tale tracciamento. Inoltre, devono informare gli utenti su come i loro dati vengono utilizzati per la pubblicità mirata e chi ha finanziato gli annunci."
},
# Question 7 from GDPR
{
    'law': 'gdpr',
    'question': "Quali sono le basi giuridiche per il trattamento dei dati personali ai sensi del GDPR?",
    'answer': "Le basi giuridiche per il trattamento dei dati personali ai sensi del GDPR includono il consenso dell'interessato, l'esecuzione di un contratto, il rispetto di un obbligo legale, la protezione degli interessi vitali, l'esecuzione di un compito di interesse pubblico o l'esercizio di pubblici poteri, e il legittimo interesse del titolare del trattamento."
},
# Question 7 from AI Act
{
    'law': 'ai_act',
    'question': "Quali misure devono essere adottate per garantire la sicurezza dei sistemi di AI secondo l'AI Act?",
    'answer': "I fornitori devono garantire che i sistemi di AI siano sicuri durante tutto il loro ciclo di vita, compresa la progettazione, lo sviluppo, la verifica, e il monitoraggio. Devono anche adottare misure per prevenire e mitigare eventuali danni derivanti dall'uso improprio dei sistemi di AI, sia intenzionale che accidentale."
},
# Question 7 from DMA
{
    'law': 'dma',
    'question': "Quali sono le conseguenze per i gatekeeper che non rispettano le regole del DMA?",
    'answer': "Le conseguenze per i gatekeeper che non rispettano le regole del DMA includono multe che possono raggiungere il 10% del loro fatturato globale annuo, sanzioni periodiche di penalità, e possibili misure strutturali in caso di violazioni ripetute o gravi."
},
# Question 7 from DSA
{
    'law': 'dsa',
    'question': "Quali misure devono adottare le piattaforme per garantire la conformità con il DSA?",
    'answer': "Le piattaforme devono implementare misure di moderazione dei contenuti, garantire trasparenza nella pubblicità, eseguire valutazioni del rischio, fornire un meccanismo di ricorso, e collaborare con le autorità di vigilanza per garantire la conformità alle norme del DSA."
},

# Question 8 from GDPR
{
    'law': 'gdpr',
    'question': "Qual è il ruolo del responsabile della protezione dei dati (DPO) secondo il GDPR?",
    'answer': "Il responsabile della protezione dei dati (DPO) ha il compito di garantire che l'organizzazione rispetti le normative sul trattamento dei dati personali. Il DPO deve essere indipendente, tenuto alla riservatezza e responsabile di consigliare e monitorare l'organizzazione in merito agli obblighi del GDPR, nonché di fungere da punto di contatto con le autorità di controllo e gli interessati."
},
# Question 8 from AI Act
{
    'law': 'ai_act',
    'question': "Quali obblighi di conformità devono soddisfare i fornitori di AI ad alto rischio?",
    'answer': "I fornitori di AI ad alto rischio devono condurre valutazioni di conformità prima di mettere i loro sistemi sul mercato, mantenere una documentazione tecnica dettagliata, segnalare qualsiasi incidente grave alle autorità competenti, e implementare un sistema di gestione del rischio per mitigare i rischi associati all'uso dei loro sistemi di AI."
},
# Question 8 from DMA
{
    'law': 'dma',
    'question': "Come il DMA affronta il problema dell'autopreferenza dei servizi dei gatekeeper?",
    'answer': "Il DMA vieta ai gatekeeper di favorire i propri servizi o prodotti rispetto a quelli di terzi sulla propria piattaforma. Ad esempio, un gatekeeper non può posizionare i propri servizi in modo privilegiato nei risultati di ricerca rispetto ai servizi di altre aziende."
},
# Question 8 from DSA
{
    'law': 'dsa',
    'question': "Quali responsabilità hanno le piattaforme per il trattamento dei dati dei minori ai sensi del DSA?",
    'answer': "Le piattaforme devono adottare misure adeguate per garantire che il trattamento dei dati dei minori sia conforme al DSA, compreso il garantire la privacy e la protezione dei dati dei minori, fornendo informazioni chiare e accessibili, e offrendo opzioni per gestire il consenso e la raccolta dei dati."
},
# Question 9 from GDPR
{
    'law': 'gdpr',
    'question': "Quali sono le implicazioni del GDPR per i trattamenti automatizzati dei dati?",
    'answer': "Il GDPR impone restrizioni sui trattamenti automatizzati dei dati personali che producono effetti giuridici o influiscono significativamente sugli interessati, compresa la profilazione. Gli individui hanno il diritto di non essere soggetti a una decisione basata esclusivamente su un trattamento automatizzato senza un'adeguata base giuridica o garanzie adeguate."
},
# Question 9 from AI Act
{
    'law': 'ai_act',
    'question': "Quali obblighi devono rispettare gli utilizzatori di sistemi di AI secondo l'AI Act?",
    'answer': "Gli utilizzatori di sistemi di AI devono garantire che questi sistemi siano utilizzati in conformità con le istruzioni del fornitore, adottare misure per monitorare l'uso sicuro, e informare immediatamente il fornitore di qualsiasi incidente o malfunzionamento che possa comportare rischi per la sicurezza o i diritti fondamentali."
},
# Question 9 from DMA
{
    'law': 'dma',
    'question': "Come il DMA affronta l'interoperabilità tra i servizi di gatekeeper e terze parti?",
    'answer': "Il DMA richiede ai gatekeeper di consentire l'interoperabilità con i loro servizi principali e di garantire che le imprese terze possano accedere e utilizzare i loro servizi in modo equo, trasparente e non discriminatorio, senza imporre condizioni restrittive ingiustificate."
},
# Question 9 from DSA
{
    'law': 'dsa',
    'question': "Quali requisiti di trasparenza per gli algoritmi impone il DSA alle piattaforme online?",
    'answer': "Il DSA impone alle piattaforme online di fornire informazioni chiare e dettagliate sul funzionamento dei loro algoritmi di raccomandazione. Gli utenti devono essere informati su come i contenuti sono prioritizzati e avere la possibilità di modificare i criteri degli algoritmi utilizzati."
},
# Question 10 from GDPR
{
    'law': 'gdpr',
    'question': "Quali requisiti di sicurezza impone il GDPR per la protezione dei dati personali?",
    'answer': "Il GDPR richiede ai titolari e responsabili del trattamento di implementare misure tecniche e organizzative adeguate per garantire un livello di sicurezza adeguato al rischio, inclusa la pseudonimizzazione e la crittografia dei dati, la capacità di garantire la riservatezza, l'integrità e la disponibilità dei sistemi, e la capacità di ripristinare rapidamente la disponibilità dei dati personali in caso di incidente."
},
# Question 10 from AI Act
{
    'law': 'ai_act',
    'question': "Quali sono le sanzioni previste dall'AI Act per la non conformità?",
    'answer': "Le sanzioni per la non conformità all'AI Act possono includere multe fino a 30 milioni di euro o, nel caso di un'azienda, fino al 6% del fatturato globale annuo, a seconda di quale sia maggiore. Le sanzioni sono proporzionate alla gravità della violazione e ai rischi posti dai sistemi di AI."
},
# Question 10 from DMA
{
    'law': 'dma',
    'question': "Quali meccanismi di risoluzione delle controversie sono disponibili per gli utenti dei servizi dei gatekeeper secondo il DMA?",
    'answer': "Il DMA prevede che i gatekeeper debbano fornire meccanismi efficaci di risoluzione delle controversie per gli utenti commerciali che utilizzano i loro servizi. Questo include la possibilità di ricorso a mediatori neutrali e indipendenti, nonché la possibilità di presentare reclami alle autorità di regolamentazione competenti."
},
# Question 10 from DSA
{
    'law': 'dsa',
    'question': "Come deve avvenire la cooperazione tra le piattaforme online e le autorità nazionali ai sensi del DSA?",
    'answer': "Le piattaforme online devono cooperare con le autorità nazionali di vigilanza fornendo informazioni necessarie per l'applicazione del DSA, rispondendo prontamente alle richieste di informazioni e attuando le decisioni delle autorità. Inoltre, devono nominare un rappresentante legale nell'Unione Europea, se non sono stabilite nell'UE."
},
# Question 11 from GDPR
{
    'law': 'gdpr',
    'question': "Che cos'è una valutazione d'impatto sulla protezione dei dati (DPIA) e quando è richiesta secondo il GDPR?",
    'answer': "Una valutazione d'impatto sulla protezione dei dati (DPIA) è un processo volto a descrivere il trattamento dei dati personali, valutare la necessità e la proporzionalità del trattamento, e gestire i rischi per i diritti e le libertà degli individui. È richiesta quando un trattamento può comportare un rischio elevato per i diritti e le libertà delle persone fisiche, in particolare nei casi di uso di nuove tecnologie o profilazione sistematica."
},
# Question 11 from AI Act
{
    'law': 'ai_act',
    'question': "Quali responsabilità hanno i distributori di sistemi di AI secondo l'AI Act?",
    'answer': "I distributori di sistemi di AI devono assicurarsi che i sistemi che distribuiscono siano conformi all'AI Act, fornendo informazioni chiare e accurate agli utenti, e adottando misure per mitigare i rischi associati all'uso dei sistemi. Inoltre, devono cooperare con le autorità di sorveglianza e segnalare eventuali non conformità."
},
# Question 11 from DMA
{
    'law': 'dma',
    'question': "Quali sono le conseguenze per i gatekeeper che abusano del loro potere di mercato secondo il DMA?",
    'answer': "I gatekeeper che abusano del loro potere di mercato possono essere soggetti a sanzioni severe, compresi ordini di cessazione delle pratiche abusive, multe significative, e in casi estremi, misure strutturali come la separazione delle attività aziendali per garantire il rispetto delle regole del DMA."
},
# Question 11 from DSA
{
    'law': 'dsa',
    'question': "Come devono gestire le piattaforme online la trasparenza sui termini e condizioni secondo il DSA?",
    'answer': "Le piattaforme online devono fornire termini e condizioni chiari e comprensibili per gli utenti, spiegando in modo trasparente le politiche di utilizzo dei dati, le regole di moderazione dei contenuti, e i diritti e le responsabilità degli utenti. Devono inoltre aggiornare i termini e condizioni in modo regolare e informare gli utenti di eventuali cambiamenti."
},
# Question 12 from GDPR
{
    'law': 'gdpr',
    'question': "Quali diritti hanno gli interessati di opporsi al trattamento dei dati personali secondo il GDPR?",
    'answer': "Gli interessati hanno il diritto di opporsi al trattamento dei loro dati personali in qualsiasi momento per motivi connessi alla loro situazione particolare, quando il trattamento è basato su interessi legittimi o l'esecuzione di un compito di interesse pubblico. Inoltre, hanno il diritto di opporsi al trattamento per finalità di marketing diretto."
},
# Question 12 from AI Act
{
    'law': 'ai_act',
    'question': "Come l'AI Act affronta il problema della tracciabilità nei sistemi di AI?",
    'answer': "L'AI Act richiede che i sistemi di AI ad alto rischio garantiscano la tracciabilità delle loro operazioni attraverso una documentazione accurata e verificabile, che copra tutto il ciclo di vita del sistema, dalla progettazione e sviluppo fino all'implementazione e monitoraggio. Ciò è necessario per garantire la trasparenza e la responsabilità degli sviluppatori e degli utilizzatori."
},
# Question 12 from DMA
{
    'law': 'dma',
    'question': "Quali pratiche sono vietate per i gatekeeper per evitare la concorrenza sleale secondo il DMA?",
    'answer': "Il DMA vieta ai gatekeeper di utilizzare pratiche che impediscono o limitano la concorrenza, come limitare l'accesso ai dati generati dagli utenti, imporre condizioni di accesso irragionevoli ai loro servizi di base, o utilizzare i dati degli utenti commerciali per competere contro di loro."
},
# Question 12 from DSA
{
    'law': 'dsa',
    'question': "Quali obblighi di reportistica hanno le piattaforme molto grandi ai sensi del DSA?",
    'answer': "Le piattaforme molto grandi devono pubblicare rapporti annuali sulla moderazione dei contenuti, fornire dettagli su come gestiscono i rischi sistemici, come l'uso abusivo delle loro piattaforme, e segnalare alle autorità competenti qualsiasi minaccia significativa per la sicurezza pubblica o la salute."
},
# Question 13 from GDPR
{
    'law': 'gdpr',
    'question': "Cosa prevede il GDPR riguardo alla notificazione delle violazioni dei dati personali?",
    'answer': "Il GDPR richiede che i titolari del trattamento notifichino le violazioni dei dati personali all'autorità di controllo competente senza ingiustificato ritardo e, se possibile, entro 72 ore dal momento in cui ne sono venuti a conoscenza, a meno che la violazione dei dati personali non sia improbabile che comporti un rischio per i diritti e le libertà delle persone fisiche."
},
# Question 13 from AI Act
{
    'law': 'ai_act',
    'question': "Quali requisiti di documentazione devono rispettare i fornitori di AI secondo l'AI Act?",
    'answer': "I fornitori di sistemi di AI ad alto rischio devono mantenere una documentazione tecnica dettagliata che descriva le specifiche del sistema, il suo scopo previsto, i dati di addestramento utilizzati, e le misure adottate per mitigare i rischi. La documentazione deve essere accurata, completa e aggiornata, e deve essere messa a disposizione delle autorità competenti su richiesta."
},
# Question 13 from DMA
{
    'law': 'dma',
    'question': "Come il DMA promuove l'innovazione e la concorrenza nel mercato digitale?",
    'answer': "Il DMA promuove l'innovazione e la concorrenza vietando pratiche sleali dei gatekeeper che limitano la scelta dei consumatori e soffocano la concorrenza, obbligando i gatekeeper a garantire l'accesso equo ai loro servizi e piattaforme, e sostenendo l'interoperabilità e l'apertura dei mercati digitali."
},
# Question 13 from DSA
{
    'law': 'dsa',
    'question': "Quali sono le responsabilità delle piattaforme online riguardo alla gestione delle denunce degli utenti ai sensi del DSA?",
    'answer': "Le piattaforme online devono fornire meccanismi facilmente accessibili per consentire agli utenti di presentare denunce contro contenuti illegali o contro decisioni di moderazione dei contenuti. Devono gestire queste denunce in modo tempestivo, fornire risposte motivate e offrire la possibilità di ricorso in caso di disaccordo con la decisione presa."
},
# Question 14 from GDPR
{
    'law': 'gdpr',
    'question': "Qual è il ruolo dell'autorità di controllo secondo il GDPR?",
    'answer': "L'autorità di controllo è responsabile di monitorare l'applicazione del GDPR per proteggere i diritti e le libertà fondamentali delle persone fisiche in relazione al trattamento dei dati personali e facilitare il libero flusso dei dati personali all'interno dell'Unione Europea. L'autorità ha il potere di condurre indagini, emettere avvertimenti e multe, e di ordinare la sospensione delle attività di trattamento."
},
# Question 14 from AI Act
{
    'law': 'ai_act',
    'question': "Come deve essere garantita la supervisione umana dei sistemi di AI secondo l'AI Act?",
    'answer': "L'AI Act richiede che i sistemi di AI ad alto rischio siano progettati e sviluppati in modo da permettere un'adeguata supervisione umana, per ridurre i rischi per la salute, la sicurezza e i diritti fondamentali. Gli utenti dei sistemi di AI devono essere in grado di interpretare e comprendere i risultati del sistema e avere la possibilità di intervenire e disattivare il sistema se necessario."
},
# Question 14 from DMA
{
    'law': 'dma',
    'question': "Quali misure correttive può adottare la Commissione Europea contro i gatekeeper secondo il DMA?",
    'answer': "La Commissione Europea può adottare una serie di misure correttive contro i gatekeeper che non rispettano il DMA, tra cui ordini di cessazione delle pratiche sleali, multe significative fino al 10% del fatturato globale annuo, e, in casi estremi, misure strutturali come la separazione delle attività aziendali."
},
# Question 14 from DSA
{
    'law': 'dsa',
    'question': "Qual è il ruolo delle autorità di coordinamento digitale ai sensi del DSA?",
    'answer': "Le autorità di coordinamento digitale sono responsabili di garantire l'applicazione uniforme delle regole del DSA all'interno degli Stati membri. Devono collaborare con altre autorità nazionali e la Commissione Europea per monitorare il rispetto delle norme, gestire le denunce e adottare misure di enforcement contro le piattaforme che violano le regole del DSA."
},
# Question 15 from GDPR
{
    'law': 'gdpr',
    'question': "Quali diritti hanno gli interessati riguardo alla rettifica dei propri dati personali?",
    'answer': "Gli interessati hanno il diritto di ottenere dal titolare del trattamento la rettifica dei dati personali inesatti che li riguardano senza ingiustificato ritardo. Inoltre, hanno il diritto di ottenere l'integrazione dei dati personali incompleti, anche fornendo una dichiarazione integrativa."
},
# Question 15 from AI Act
{
    'law': 'ai_act',
    'question': "Quali sono le disposizioni dell'AI Act per l'uso dei sistemi di riconoscimento biometrico remoto?",
    'answer': "L'AI Act prevede restrizioni rigorose sull'uso dei sistemi di riconoscimento biometrico remoto in tempo reale per scopi di applicazione della legge, limitandoli a casi eccezionali come la ricerca di minori scomparsi, la prevenzione di minacce terroristiche o la localizzazione di persone ricercate per crimini gravi, e solo con l'autorizzazione giudiziaria o regolamentare."
},
# Question 15 from DMA
{
    'law': 'dma',
    'question': "Come il DMA regola l'accesso dei terzi ai dati generati dagli utenti sulle piattaforme dei gatekeeper?",
    'answer': "Il DMA impone ai gatekeeper di consentire l'accesso equo e non discriminatorio ai dati generati dagli utenti sulle loro piattaforme. Devono fornire agli utenti commerciali l'accesso ai dati pertinenti in un formato leggibile da macchina e garantire che i terzi possano utilizzare i dati per sviluppare e offrire prodotti e servizi concorrenti."
},
# Question 15 from DSA
{
    'law': 'dsa',
    'question': "Come devono essere segnalati gli incidenti di sicurezza da parte delle piattaforme online ai sensi del DSA?",
    'answer': "Le piattaforme online devono notificare immediatamente alle autorità di coordinamento digitale e, se del caso, alle autorità nazionali competenti qualsiasi incidente di sicurezza che possa influire significativamente sulla sicurezza dei dati degli utenti o compromettere la fiducia nel loro servizio. Devono anche adottare misure per mitigare gli effetti dell'incidente."
},
# Question 16 from GDPR
{
    'law': 'gdpr',
    'question': "Come vengono trattati i dati personali dei bambini secondo il GDPR?",
    'answer': "Il GDPR prevede disposizioni speciali per il trattamento dei dati personali dei bambini, riconoscendo che meritano una protezione specifica. In particolare, il consenso al trattamento dei dati personali dei bambini deve essere fornito o autorizzato dal titolare della responsabilità genitoriale quando il bambino ha meno di 16 anni (o un'età inferiore se stabilita dagli Stati membri)."
},
# Question 16 from AI Act
{
    'law': 'ai_act',
    'question': "Che ruolo ha la valutazione della conformità nell'AI Act?",
    'answer': "La valutazione della conformità è un processo attraverso il quale i fornitori di sistemi di AI ad alto rischio devono dimostrare che i loro prodotti rispettano i requisiti dell'AI Act. La valutazione deve essere effettuata prima dell'immissione sul mercato e può includere test, ispezioni e audit condotti da organismi notificati o da valutazioni interne."
},
# Question 16 from DMA
{
    'law': 'dma',
    'question': "In che modo il DMA supporta la portabilità dei dati per gli utenti?",
    'answer': "Il DMA richiede che i gatekeeper forniscano agli utenti la possibilità di trasferire i propri dati personali e non personali a fornitori di servizi terzi in modo agevole e sicuro, facilitando la portabilità dei dati e promuovendo la concorrenza nei mercati digitali."
},
# Question 16 from DSA
{
    'law': 'dsa',
    'question': "Quali misure devono adottare le piattaforme per garantire che i loro termini e condizioni siano comprensibili per gli utenti ai sensi del DSA?",
    'answer': "Le piattaforme devono garantire che i loro termini e condizioni siano redatti in un linguaggio chiaro e semplice, facilmente comprensibile per tutti gli utenti, compresi i bambini. Devono inoltre garantire che i termini siano facilmente accessibili e trasparenti riguardo alle politiche di contenuto, ai diritti degli utenti e alle procedure di segnalazione."
},
# Question 17 from GDPR
{
    'law': 'gdpr',
    'question': "Quali criteri devono essere considerati per determinare la liceità del trattamento dei dati ai sensi del GDPR?",
    'answer': "Il GDPR richiede che il trattamento dei dati sia lecito solo se soddisfa almeno una delle basi giuridiche stabilite, come il consenso dell'interessato, l'esecuzione di un contratto, l'adempimento di un obbligo legale, la protezione degli interessi vitali, l'esecuzione di un compito di interesse pubblico o l'esercizio di pubblici poteri, o il legittimo interesse del titolare del trattamento."
},
# Question 17 from AI Act
{
    'law': 'ai_act',
    'question': "Qual è il ruolo delle autorità di sorveglianza secondo l'AI Act?",
    'answer': "Le autorità di sorveglianza hanno il compito di monitorare e garantire l'applicazione dell'AI Act, fornendo orientamenti e raccomandazioni, indagando sulle violazioni della normativa, e cooperando con le altre autorità nazionali ed europee per garantire la coerenza nell'applicazione delle regole e delle sanzioni."
},
# Question 17 from DMA
{
    'law': 'dma',
    'question': "Quali requisiti di interoperabilità tra i servizi di gatekeeper e i servizi di terze parti impone il DMA?",
    'answer': "Il DMA impone ai gatekeeper di garantire l'interoperabilità tra i loro servizi di piattaforma di base e i servizi di terze parti, in modo equo, trasparente e non discriminatorio, senza impedire l'accesso alle funzionalità principali o all'esperienza utente."
},
# Question 17 from DSA
{
    'law': 'dsa',
    'question': "Quali sono le responsabilità delle piattaforme molto grandi in termini di valutazione del rischio ai sensi del DSA?",
    'answer': "Le piattaforme molto grandi devono effettuare valutazioni del rischio sistemico associato ai loro servizi almeno una volta all'anno. Devono identificare i rischi legati alla diffusione di contenuti illegali, alla disinformazione e ad altri potenziali danni sociali, e adottare misure per mitigare questi rischi."
},
# Question 18 from GDPR
{
    'law': 'gdpr',
    'question': "Come il GDPR gestisce il consenso per il trattamento dei dati personali?",
    'answer': "Il GDPR stabilisce che il consenso deve essere fornito liberamente, specificamente, informato e inequivocabile. Il consenso deve essere espresso attraverso un'azione affermativa chiara, e l'interessato ha il diritto di revocarlo in qualsiasi momento. Il consenso non può essere presunto o implicito dall'inattività dell'interessato."
},
# Question 18 from AI Act
{
    'law': 'ai_act',
    'question': "Quali requisiti di qualità dei dati sono stabiliti dall'AI Act per i sistemi di AI?",
    'answer': "L'AI Act richiede che i dati utilizzati per addestrare, convalidare e testare i sistemi di AI ad alto rischio siano pertinenti, rappresentativi, privi di errori, e aggiornati, per garantire che i sistemi funzionino in modo affidabile e sicuro. I fornitori devono implementare processi per verificare la qualità dei dati utilizzati."
},
# Question 18 from DMA
{
    'law': 'dma',
    'question': "Quali sono gli obblighi dei gatekeeper per garantire una concorrenza leale ai sensi del DMA?",
    'answer': "I gatekeeper devono garantire che i loro servizi siano offerti in modo non discriminatorio a tutte le aziende e utenti. Devono evitare pratiche come l'autopreferenza, l'imposizione di condizioni inique, o il blocco dell'accesso a dati essenziali, e devono collaborare con le autorità di regolamentazione per garantire una concorrenza leale."
},
# Question 18 from DSA
{
    'law': 'dsa',
    'question': "Come devono le piattaforme online garantire la trasparenza nella moderazione dei contenuti secondo il DSA?",
    'answer': "Le piattaforme devono pubblicare rapporti regolari sulla moderazione dei contenuti, spiegando le decisioni prese riguardo alla rimozione o alla limitazione di contenuti. Devono inoltre fornire agli utenti informazioni chiare e facilmente accessibili sui criteri utilizzati per moderare i contenuti e offrire meccanismi di ricorso equi."
},
# Question 19 from GDPR
{
    'law': 'gdpr',
    'question': "Come il GDPR affronta il trattamento dei dati per finalità di ricerca scientifica?",
    'answer': "Il GDPR consente deroghe ai diritti degli interessati per il trattamento dei dati personali a fini di ricerca scientifica, purché siano messe in atto garanzie adeguate, come la pseudonimizzazione, per proteggere i diritti e le libertà degli individui. Tuttavia, gli interessati conservano il diritto di opporsi a tale trattamento."
},
# Question 19 from AI Act
{
    'law': 'ai_act',
    'question': "Quali sono i requisiti di conformità per i sistemi di AI utilizzati nei prodotti regolamentati da altre normative dell'UE?",
    'answer': "I sistemi di AI utilizzati nei prodotti regolamentati da altre normative dell'UE devono rispettare sia i requisiti dell'AI Act sia le normative settoriali specifiche. I fornitori devono garantire che i sistemi siano sicuri e conformi agli standard tecnici e normativi applicabili e fornire una documentazione dettagliata a supporto della conformità."
},
# Question 19 from DMA
{
    'law': 'dma',
    'question': "Quali diritti hanno le imprese utenti dei servizi dei gatekeeper secondo il DMA?",
    'answer': "Le imprese utenti dei servizi dei gatekeeper hanno il diritto di accedere ai dati generati dalle loro attività sulla piattaforma, di utilizzare servizi alternativi e di contestare le decisioni del gatekeeper attraverso meccanismi di risoluzione delle controversie neutrali e indipendenti."
},
# Question 19 from DSA
{
    'law': 'dsa',
    'question': "Quali sono gli obblighi di notifica per le piattaforme che rimuovono o disabilitano l'accesso ai contenuti secondo il DSA?",
    'answer': "Le piattaforme devono notificare agli utenti quando rimuovono o disabilitano l'accesso ai contenuti, fornendo una spiegazione chiara del motivo, le basi giuridiche, e le informazioni sui diritti di ricorso dell'utente contro la decisione. Devono anche offrire un processo di revisione rapido e trasparente."
},
# Question 20 from GDPR
{
    'law': 'gdpr',
    'question': "Quali misure tecniche e organizzative devono essere adottate per garantire la sicurezza dei dati personali ai sensi del GDPR?",
    'answer': "Il GDPR richiede che i titolari e i responsabili del trattamento implementino misure tecniche e organizzative adeguate per garantire un livello di sicurezza adeguato al rischio, inclusa la pseudonimizzazione e la crittografia dei dati, misure per garantire la riservatezza, l'integrità, la disponibilità e la resilienza dei sistemi e dei servizi, e la capacità di ripristinare rapidamente la disponibilità dei dati in caso di incidente."
},
# Question 20 from AI Act
{
    'law': 'ai_act',
    'question': "Come deve avvenire la gestione del ciclo di vita dei sistemi di AI secondo l'AI Act?",
    'answer': "La gestione del ciclo di vita dei sistemi di AI richiede che i fornitori garantiscano la conformità alle norme durante tutte le fasi, dalla progettazione e sviluppo alla distribuzione e utilizzo, fino alla disattivazione. Devono monitorare continuamente le prestazioni e aggiornare i sistemi per gestire nuovi rischi o vulnerabilità."
},
# Question 20 from DMA
{
    'law': 'dma',
    'question': "Come il DMA facilita la contestazione delle pratiche sleali dei gatekeeper da parte degli utenti commerciali?",
    'answer': "Il DMA richiede che i gatekeeper forniscano meccanismi di reclamo e risoluzione delle controversie equi e trasparenti per gli utenti commerciali. Questi meccanismi includono la possibilità di mediazione e arbitrato da parte di terzi indipendenti e l'accesso a ricorsi legali presso le autorità competenti."
},
# Question 20 from DSA
{
    'law': 'dsa',
    'question': "Quali sono le implicazioni per le piattaforme che non rispettano il DSA?",
    'answer': "Le piattaforme che non rispettano il DSA possono essere soggette a sanzioni amministrative, che possono includere multe significative fino al 6% del loro fatturato globale annuo. Le autorità di coordinamento digitale possono inoltre imporre altre misure correttive, come ordini di cessazione o restrizioni operative."
}
]

    # Update the laws_info dictionary for GDPR, AI Act, DMA, and DSA
laws_info = {
    'gdpr': {
        'file_path': '/kaggle/input/chroma-db-italian/Italian_gdpr.html',
        'collection_name': 'embeddings_gdpr',
        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'gdpr']
    },
    'ai_act': {
        'file_path': '/kaggle/input/chroma-db-italian/italian_AI_act.html',
        'collection_name': 'embeddings_ai_act',
        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'ai_act']
    },
    'dma': {
        'file_path': '/kaggle/input/chroma-db-italian/italian_dma.html',
        'collection_name': 'embeddings_dma',
        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'dma']
    },
    'dsa': {
        'file_path': '/kaggle/input/chroma-db-italian/italian_dsa.html',
        'collection_name': 'embeddings_dsa',
        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'dsa']
    },
}



# Function to handle multiple queries and print results for all laws
def embed_and_query_all_laws(laws_info, model_norm, collections, top_k=1):
    for law, info in laws_info.items():
        print(f"\nQuerying {law.upper()} collection:")
        
        for qa in info['questions_answers']:
            query = qa['question']
            reference_answer = qa['answer']

            # Embed and query
            results = embed_and_query(query, model_norm, collections[law], top_k)

            if results and 'documents' in results and results['documents']:
                retrieved_text = results['documents'][0][0]  # Assuming it's a list of lists
                
                # Generate embeddings using BERT for cosine similarity
                retrieved_embedding = generate_bert_embedding(retrieved_text, tokenizer, model)
                reference_embedding = generate_bert_embedding(reference_answer, tokenizer, model)

                # Calculate cosine similarity using BERT embeddings
                cosine_sim = calculate_cosine_similarity(reference_embedding, retrieved_embedding)

                # Calculate semantic similarity using Sentence-Transformers model
                semantic_sim = calculate_semantic_similarity(reference_answer, retrieved_text, semantic_model)

                # Print results
                chunk_id = results['ids'][0][0]  # Accessing the first element in the list of IDs
                print(f"Query: {query}")
                print(f"Retrieved chunk {chunk_id.split('_')[-1]} from {law.upper()}:")
                print(f"Retrieved text: {retrieved_text}")
                print(f"Reference answer: {reference_answer}")
                print(f"Cosine Similarity: {cosine_sim:.4f}")
                print(f"Semantic Similarity: {semantic_sim:.4f}")
                print("----\n")
            else:
                print(f"No valid results found for query: {query} in {law.upper()}.")

# Query and print results for all questions across GDPR, AI Act, DMA, and DSA
embed_and_query_all_laws(laws_info, model_norm, collections, top_k=1)


Querying GDPR collection:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Qual è il diritto fondamentale relativo al trattamento dei dati personali secondo la Carta dei diritti fondamentali dell'Unione Europea?
Retrieved chunk 180 from GDPR:
Retrieved text: (167) Al fine di garantire condizioni uniformi di esecuzione del presente regolamento, dovrebbero essere attribuite alla Commissione competenze di esecuzione ove previsto dal presente regolamento. Tali competenze dovrebbero essere esercitate conformemente al regolamento (UE) n. 182/2011 del Parlamento europeo e del Consiglio.A tal fine, la Commissione dovrebbe contemplare misure specifiche per le micro, piccole e medie imprese.
Reference answer: La protezione delle persone fisiche in relazione al trattamento dei dati personali è un diritto fondamentale. L'articolo 8(1) della Carta dei diritti fondamentali dell'Unione Europea ('la Carta') e l'articolo 16(1) del Trattato sul funzionamento dell'Unione Europea (TFUE) stabiliscono che ogni persona ha il diritto alla protezione dei dati personali che la 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Quali sono gli obblighi di notifica per le piattaforme che rimuovono o disabilitano l'accesso ai contenuti secondo il DSA?
Retrieved chunk 235 from DSA:
Retrieved text: Il presente articolo lascia impregiudicata la possibilità, conformemente all'ordinamento giuridico dello Stato membro, che un'autorità giudiziaria o amministrativa esiga che il prestatore del servizio impedisca o ponga fine a una violazione. Articolo 5 Memorizzazione temporanea 1.
Reference answer: Le piattaforme devono notificare agli utenti quando rimuovono o disabilitano l'accesso ai contenuti, fornendo una spiegazione chiara del motivo, le basi giuridiche, e le informazioni sui diritti di ricorso dell'utente contro la decisione. Devono anche offrire un processo di revisione rapido e trasparente.
Cosine Similarity: 0.9993
Semantic Similarity: 0.9402
----



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Quali sono le implicazioni per le piattaforme che non rispettano il DSA?
Retrieved chunk 316 from DSA:
Retrieved text: La Commissione, in cooperazione con i coordinatori dei servizi digitali, può emanare orientamenti sull'applicazione del paragrafo 1 in relazione a rischi concreti, con l'obiettivo specifico di presentare le migliori pratiche e raccomandare eventuali misure, tenendo debitamente conto delle possibili conseguenze di tali misure sui diritti fondamentali di tutte le parti interessate sanciti dalla Carta. Nell'elaborazione di tali orientamenti la Commissione organizza consultazioni pubbliche. Articolo 36 Meccanismo di risposta alle crisi 1.
Reference answer: Le piattaforme che non rispettano il DSA possono essere soggette a sanzioni amministrative, che possono includere multe significative fino al 6% del loro fatturato globale annuo. Le autorità di coordinamento digitale possono inoltre imporre altre misure correttive, come ordini di cessazione o restrizioni operative

# summarization of the 20

In [11]:
from collections import defaultdict
from transformers import pipeline

# Load the Hugging Face su mmarization model
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

# Convert integrated questions and answers to a dictionary format
def create_laws_info(questions_answers_list):
    """
    Dynamically creates a dictionary of laws and their questions and answers.

    Args:
    - questions_answers_list (list): List of dictionaries containing law, question, and answer data.

    Returns:
    
    - laws_info (dict): A dictionary with each law as a key, and associated questions and answers.
    """
    laws_info = defaultdict(lambda: {'questions_answers': []})

    # Group questions and answers by law
    for entry in questions_answers_list:
        law = entry['law']
        laws_info[law]['questions_answers'].append(entry)

    return laws_info

# Automatically construct laws_info dictionary
laws_info = create_laws_info(integrated_questions_answers)

# Function to summarize a given text using Hugging Face model with retry logic
def summarize_text_huggingface_with_retry(text, max_length=350, min_length=100, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Generate summary
            summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
            return summary[0]['summary_text']
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                print("Retrying...")
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                print("Max retries reached. Moving to next text.")
                return None

# Function to handle summarization of chunks for all laws using Hugging Face model
def summarize_all_laws_huggingface(laws_info):
    for law, info in laws_info.items():
        print(f"\nProcessing summaries for {law.upper()}...")

        for qa in info['questions_answers']:
            question = qa['question']
            retrieved_text = qa['answer']  # Assuming 'answer' is the text to be summarized

            print(f"Processing {law.upper()} - Question: {question}")

            # Generate a summary for each retrieved text using the Hugging Face model
            summary = summarize_text_huggingface_with_retry(retrieved_text)
            if summary:
                qa['summary'] = summary  # Store the summary for comparison later
                print(f"Summary for {law.upper()} - Question: {question}:\n{summary}\n----\n")
            else:
                qa['summary'] = None
                print(f"Failed to generate a summary for {law.upper()} - Question: {question}\n")

# Run the summarization for all laws
summarize_all_laws_huggingface(laws_info)



config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Summary for DSA - Question: Quali sono le responsabilità delle piattaforme molto grandi in termini di valutazione del rischio ai sensi del DSA?:
Le piattaforme molto grandi devono effettuare valutazioni del rischio sistemico associato ai loro servizi almeno una volta all'anno. Devono identificare i rischi legati alla diffusione di contenuti illegali, alla disinformazione e ad altri potenziali danni sociali, and adottare misure per mitigare these rischi.
----

Processing DSA - Question: Come devono le piattaforme online garantire la trasparenza nella moderazione dei contenuti secondo il DSA?


Your max_length is set to 350, but your input_length is only 108. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)


Summary for DSA - Question: Come devono le piattaforme online garantire la trasparenza nella moderazione dei contenuti secondo il DSA?:
Le piattaforme devono pubblicare rapporti regolari sulla moderazione dei contenuti. Devono fornire agli utenti informazioni chiare e facilmente accessibili sui criteri utilizzati per moderare i contenuti and offrire meccanismi di ricorso equi. i rapporti prese riguardo alla rimozione o alla limitazione.
----

Processing DSA - Question: Quali sono gli obblighi di notifica per le piattaforme che rimuovono o disabilitano l'accesso ai contenuti secondo il DSA?


Your max_length is set to 350, but your input_length is only 101. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Summary for DSA - Question: Quali sono gli obblighi di notifica per le piattaforme che rimuovono o disabilitano l'accesso ai contenuti secondo il DSA?:
Le piattaforme devono notificare agli utenti quando rimuovono o disabilitano l'accesso ai contenuti. Devono offrire un processo di revisione rapido e trasparente.   Â   .   i giuridiche, le basi giuri, and le informazioni sui diritti di ricorso dell'tutente contro la decisione.
----

Processing DSA - Question: Quali sono le implicazioni per le piattaforme che non rispettano il DSA?
Summary for DSA - Question: Quali sono le implicazioni per le piattaforme che non rispettano il DSA?:
Le piattaforme che non rispettano il DSA can be soggette a sanzioni amministrative. Le autorità di coordinamento digitale possono inoltre imporre misure correttive, such as ordini di cessazione o restrizioni operative. Le piattica can be included in multe significative fino al 6% del loro fatturato globale annuo.
----



# compare the summaries of the 20 with the reference answers

In [12]:
# Function to compare the summaries with reference answers
def compare_summaries_with_reference(laws_info, tokenizer, model, semantic_model):
    # Initialize the similarities dictionary
    similarities = {law: {'cosine': [], 'semantic': []} for law in laws_info}

    for law, info in laws_info.items():
        print(f"\nProcessing law: {law.upper()}")

        for qa in info['questions_answers']:
            query = qa['question']
            reference_answer = qa['answer']
            summary = qa.get('summary')  # Access the stored summary from summarization

            if summary:
                # Step 1: Generate embeddings for cosine similarity
                reference_embedding = generate_bert_embedding(reference_answer, tokenizer, model)
                summary_embedding = generate_bert_embedding(summary, tokenizer, model)

                # Step 2: Calculate cosine similarity
                cosine_sim = calculate_cosine_similarity(reference_embedding, summary_embedding)

                # Step 3: Calculate semantic similarity
                semantic_sim = calculate_semantic_similarity(reference_answer, summary, semantic_model)

                # Step 4: Store the cosine and semantic similarities
                similarities[law]['cosine'].append(cosine_sim)
                similarities[law]['semantic'].append(semantic_sim)

                # Step 5: Print results
                print(f"Query: {query}")
                print(f"Summary: {summary}")
                print(f"Reference answer: {reference_answer}")
                print(f"Cosine Similarity: {cosine_sim:.4f}")
                print(f"Semantic Similarity: {semantic_sim:.4f}")
                print("----\n")
            else:
                print(f"No summary available for query: {query} in {law.upper()}")

    return similarities

# Example usage:
# Call the comparison function and store the similarities
similarities = compare_summaries_with_reference(laws_info, tokenizer, model, semantic_model)

# You can then process the similarities further, for example, calculating averages.


 # Calculate and print the averages
print("\nCalculated Averages:")
for law in similarities:
    cosine_values = similarities[law]['cosine']
    semantic_values = similarities[law]['semantic']

        # Calculate average cosine similarity if there are values
    if cosine_values:
        avg_cosine = sum(cosine_values) / len(cosine_values)
    else:
        avg_cosine = None

        # Calculate average semantic similarity if there are values
    if semantic_values:
        avg_semantic = sum(semantic_values) / len(semantic_values)
    else:
        avg_semantic = None

        # Print the results
    if avg_cosine is not None and avg_semantic is not None:
        print(f"{law.upper()} Average Cosine Similarity: {avg_cosine:.4f}")
        print(f"{law.upper()} Average Semantic Similarity: {avg_semantic:.4f}")
    else:
        print(f"No valid similarities found for {law.upper()}")


Processing law: GDPR


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Qual è il diritto fondamentale relativo al trattamento dei dati personali secondo la Carta dei diritti fondamentali dell'Unione Europea?
Summary: La protezione delle persone fisiche in relazione al trattamento dei dati personali is un diritto fondamentale. La Carta dei diritti fondamentali dell'tUnione Europea (TFUE) and the Trattato sul funzionamento ofll'Unione Europeana (TUE) stipulate that ogni persona ha il diritto alla protezioni dei data personali.
Reference answer: La protezione delle persone fisiche in relazione al trattamento dei dati personali è un diritto fondamentale. L'articolo 8(1) della Carta dei diritti fondamentali dell'Unione Europea ('la Carta') e l'articolo 16(1) del Trattato sul funzionamento dell'Unione Europea (TFUE) stabiliscono che ogni persona ha il diritto alla protezione dei dati personali che la riguardano. Questo Regolamento mira a contribuire alla realizzazione di uno spazio di libertà, sicurezza e giustizia e di un'unione economica, al progresso 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Quali sono gli obblighi di notifica per le piattaforme che rimuovono o disabilitano l'accesso ai contenuti secondo il DSA?
Summary: Le piattaforme devono notificare agli utenti quando rimuovono o disabilitano l'accesso ai contenuti. Devono offrire un processo di revisione rapido e trasparente.   Â   .   i giuridiche, le basi giuri, and le informazioni sui diritti di ricorso dell'tutente contro la decisione.
Reference answer: Le piattaforme devono notificare agli utenti quando rimuovono o disabilitano l'accesso ai contenuti, fornendo una spiegazione chiara del motivo, le basi giuridiche, e le informazioni sui diritti di ricorso dell'utente contro la decisione. Devono anche offrire un processo di revisione rapido e trasparente.
Cosine Similarity: 0.9986
Semantic Similarity: 0.9602
----



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Quali sono le implicazioni per le piattaforme che non rispettano il DSA?
Summary: Le piattaforme che non rispettano il DSA can be soggette a sanzioni amministrative. Le autorità di coordinamento digitale possono inoltre imporre misure correttive, such as ordini di cessazione o restrizioni operative. Le piattica can be included in multe significative fino al 6% del loro fatturato globale annuo.
Reference answer: Le piattaforme che non rispettano il DSA possono essere soggette a sanzioni amministrative, che possono includere multe significative fino al 6% del loro fatturato globale annuo. Le autorità di coordinamento digitale possono inoltre imporre altre misure correttive, come ordini di cessazione o restrizioni operative.
Cosine Similarity: 0.9990
Semantic Similarity: 0.9663
----


Calculated Averages:
GDPR Average Cosine Similarity: 0.9982
GDPR Average Semantic Similarity: 0.9452
AI_ACT Average Cosine Similarity: 0.9982
AI_ACT Average Semantic Similarity: 0.9452
DMA Average Cos

# averages of cosine and semantic similarities for each law of the 20 questions

In [13]:
# Function to compare the summaries with reference answers
def compare_summaries_with_reference(laws_info, tokenizer, model, semantic_model):
    # Dictionary to store similarities for each law
    similarities = {law: {'cosine': [], 'semantic': []} for law in laws_info}

    for law, info in laws_info.items():
        print(f"\nProcessing law: {law.upper()}")

        for qa in info['questions_answers']:
            query = qa['question']
            reference_answer = qa['answer']
            summary = qa.get('summary')  # Access the stored summary from summarization

            if summary:
                # Step 1: Generate embeddings for cosine similarity
                reference_embedding = generate_bert_embedding(reference_answer, tokenizer, model)
                summary_embedding = generate_bert_embedding(summary, tokenizer, model)

                # Step 2: Calculate cosine similarity
                cosine_sim = calculate_cosine_similarity(reference_embedding, summary_embedding)

                # Step 3: Calculate semantic similarity
                semantic_sim = calculate_semantic_similarity(reference_answer, summary, semantic_model)

                # Step 4: Store the cosine and semantic similarities
                similarities[law]['cosine'].append(cosine_sim)
                similarities[law]['semantic'].append(semantic_sim)

                # Step 5: Print results for each comparison
                print(f"Query: {query}")
                print(f"Summary: {summary}")
                print(f"Reference answer: {reference_answer}")
                print(f"Cosine Similarity: {cosine_sim:.4f}")
                print(f"Semantic Similarity: {semantic_sim:.4f}")
                print("----\n")
            else:
                print(f"No summary available for query: {query} in {law.upper()}")

    return similarities

# Function to calculate and print averages for cosine and semantic similarities
def calculate_and_print_averages(similarities):
    print("\nCalculated Averages:")
    
    for law, similarity_data in similarities.items():
        cosine_values = similarity_data['cosine']
        semantic_values = similarity_data['semantic']

        # Calculate average cosine similarity if values are present
        if cosine_values:
            avg_cosine = sum(cosine_values) / len(cosine_values)
        else:
            avg_cosine = None
        
        # Calculate average semantic similarity if values are present
        if semantic_values:
            avg_semantic = sum(semantic_values) / len(semantic_values)
        else:
            avg_semantic = None

        # Print the results
        if avg_cosine is not None and avg_semantic is not None:
            print(f"{law.upper()} Average Cosine Similarity: {avg_cosine:.4f}")
            print(f"{law.upper()} Average Semantic Similarity: {avg_semantic:.4f}")
        else:
            print(f"No valid similarities found for {law.upper()}")

# Example usage
# First, compare the summaries with reference answers and store the similarities
similarities = compare_summaries_with_reference(laws_info, tokenizer, model, semantic_model)

# Then, calculate and print the average cosine and semantic similarities for each law
calculate_and_print_averages(similarities)



Processing law: GDPR


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Qual è il diritto fondamentale relativo al trattamento dei dati personali secondo la Carta dei diritti fondamentali dell'Unione Europea?
Summary: La protezione delle persone fisiche in relazione al trattamento dei dati personali is un diritto fondamentale. La Carta dei diritti fondamentali dell'tUnione Europea (TFUE) and the Trattato sul funzionamento ofll'Unione Europeana (TUE) stipulate that ogni persona ha il diritto alla protezioni dei data personali.
Reference answer: La protezione delle persone fisiche in relazione al trattamento dei dati personali è un diritto fondamentale. L'articolo 8(1) della Carta dei diritti fondamentali dell'Unione Europea ('la Carta') e l'articolo 16(1) del Trattato sul funzionamento dell'Unione Europea (TFUE) stabiliscono che ogni persona ha il diritto alla protezione dei dati personali che la riguardano. Questo Regolamento mira a contribuire alla realizzazione di uno spazio di libertà, sicurezza e giustizia e di un'unione economica, al progresso 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Quali sono gli obblighi di notifica per le piattaforme che rimuovono o disabilitano l'accesso ai contenuti secondo il DSA?
Summary: Le piattaforme devono notificare agli utenti quando rimuovono o disabilitano l'accesso ai contenuti. Devono offrire un processo di revisione rapido e trasparente.   Â   .   i giuridiche, le basi giuri, and le informazioni sui diritti di ricorso dell'tutente contro la decisione.
Reference answer: Le piattaforme devono notificare agli utenti quando rimuovono o disabilitano l'accesso ai contenuti, fornendo una spiegazione chiara del motivo, le basi giuridiche, e le informazioni sui diritti di ricorso dell'utente contro la decisione. Devono anche offrire un processo di revisione rapido e trasparente.
Cosine Similarity: 0.9986
Semantic Similarity: 0.9602
----



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Quali sono le implicazioni per le piattaforme che non rispettano il DSA?
Summary: Le piattaforme che non rispettano il DSA can be soggette a sanzioni amministrative. Le autorità di coordinamento digitale possono inoltre imporre misure correttive, such as ordini di cessazione o restrizioni operative. Le piattica can be included in multe significative fino al 6% del loro fatturato globale annuo.
Reference answer: Le piattaforme che non rispettano il DSA possono essere soggette a sanzioni amministrative, che possono includere multe significative fino al 6% del loro fatturato globale annuo. Le autorità di coordinamento digitale possono inoltre imporre altre misure correttive, come ordini di cessazione o restrizioni operative.
Cosine Similarity: 0.9990
Semantic Similarity: 0.9663
----


Calculated Averages:
GDPR Average Cosine Similarity: 0.9982
GDPR Average Semantic Similarity: 0.9452
AI_ACT Average Cosine Similarity: 0.9982
AI_ACT Average Semantic Similarity: 0.9452
DMA Average Cos

# load the other 80

In [14]:
import pandas as pd

# Function to load and inspect the CSV
def inspect_csv(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path, encoding='utf-8')  # Adjust encoding if necessary
    
    # Print column names to see the headers
    print("Column Names:", df.columns)
    
    # Print the first 5 rows of the data to inspect the format
    print("\nFirst 5 Rows:\n", df.head())
    
    # Optionally, print the data types of each column
    print("\nData Types:\n", df.dtypes)

# Example: inspecting a CSV file
gdpr_csv_path = '/kaggle/input/chroma-db-italian/questions_answers-gdpr-it.csv'
inspect_csv(gdpr_csv_path)


Column Names: Index(['Domanda: Un membro di un'autorità di vigilanza può esercitare un'attività lavorativa durante il suo mandato?'], dtype='object')

First 5 Rows:
   Domanda: Un membro di un'autorità di vigilanza può esercitare un'attività lavorativa durante il suo mandato?
0  Risposta: In generale, i membri delle autorità...                                                          
1                                                NaN                                                          
2  Domanda: Qual è il ruolo di un'autorità di con...                                                          
3  Risposta: Le autorità di controllo sono fondam...                                                          
4                                                NaN                                                          

Data Types:
 Domanda: Un membro di un'autorità di vigilanza può esercitare un'attività lavorativa durante il suo mandato?    object
dtype: object


In [15]:
import pandas as pd

# Function to load and split questions and answers into separate columns
def load_and_split_questions_answers(file_path):
    df = pd.read_csv(file_path, encoding='utf-8', header=None)  # Reading without a header
    
    # Lists to store the questions and answers
    questions = []
    answers = []
    
    current_question = None  # Temporary storage for the current question
    
    for index, row in df.iterrows():
        text = row[0]  # Access the single column
        
        # Ensure that the text is a valid string (skip if NaN)
        if pd.isna(text):
            continue
        
        text = str(text)  # Convert to string just in case

        # Check if the row is a question
        if text.startswith("Domanda:"):
            current_question = text.replace("Domanda:", "").strip()  # Clean the question text
        elif text.startswith("Risposta:") and current_question:
            # If it's an answer, append both the question and answer to the lists
            answer = text.replace("Risposta:", "").strip()
            questions.append(current_question)
            answers.append(answer)
            current_question = None  # Reset after storing
    
    # Create a DataFrame with separate columns for questions and answers
    qa_df = pd.DataFrame({'question': questions, 'answer': answers})
    
    return qa_df

# List of CSV file paths for the different laws
csv_paths = {
    'gdpr': '/kaggle/input/chroma-db-italian/questions_answers-gdpr-it.csv',
    'ai_act': '/kaggle/input/chroma-db-italian/questions_answers_ai_test-it.csv',
    'dma': '/kaggle/input/chroma-db-italian/q and a italian 80 dma.csv',
    'dsa': '/kaggle/input/chroma-db-italian/questions_answers_digital_services it.csv'
}

# Dictionary to store the QA pairs DataFrames for each law
qa_dataframes = {}

# Loop over the CSV paths and process each one
for law, path in csv_paths.items():
    qa_dataframes[law] = load_and_split_questions_answers(path)
    print(f"Processed {law.upper()} law:")
    print(qa_dataframes[law].head(), "\n")

# Now, you have each law's question-answer pairs stored in the `qa_dataframes` dictionary
# You can access the GDPR data, for example, with: qa_dataframes['gdpr']


Processed GDPR law:
                                            question  \
0  Un membro di un'autorità di vigilanza può eser...   
1  Qual è il ruolo di un'autorità di controllo in...   
2  Quali sono le implicazioni legali del trasferi...   
3  In che modo il concetto di "legislazione deleg...   
4  Spiegate il concetto giuridico di "necessità u...   

                                              answer  
0  In generale, i membri delle autorità di vigila...  
1  Le autorità di controllo sono fondamentali per...  
2  Il trasferimento di dati personali in un Paese...  
3  La legislazione delegata conferisce al ramo es...  
4  La "necessità urgente di agire" indica una sit...   

Processed AI_ACT law:
                                            question  \
0  Come si applica il concetto di "modifica sosta...   
1  Può spiegare il concetto di "influenza indebit...   
2  Quali sono le implicazioni giuridiche del "pri...   
3  Quali sono le implicazioni legali di un organi...   
4  Spiega

# na rotiso ti kanei akribos

In [27]:
from collections import defaultdict
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Load Hugging Face summarization model
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum", device=0)
model_norm = SentenceTransformer('microsoft/deberta-v3-base')
# Load DeBERTa model for embedding queries (used for cosine similarity)
model_cosine = SentenceTransformer('microsoft/deberta-v3-base')

# Load a different Sentence-Transformer model for semantic similarity
model_semantic = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_and_query(query_text, model_norm, collection, top_k=1):
    query_embedding = model_norm.encode([query_text])[0]
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results

def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

def calculate_semantic_similarity(text1, text2, model_norm):
    embeddings1 = model_norm.encode([text1], convert_to_tensor=True)
    embeddings2 = model_norm.encode([text2], convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(embeddings1, embeddings2)
    return similarity.item()

# Function to summarize and compare the retrieved text with the reference answers
def summarize_and_compare_all_laws(qa_dataframes, model_norm, collections):
    # Initialize a dictionary to store cosine and semantic similarities
    similarities = defaultdict(lambda: {'cosine': [], 'semantic': []})
    
    for law, df in qa_dataframes.items():
        print(f"\nProcessing summaries for {law.upper()} law:")
        
        question_col = 'question'
        answer_col = 'answer'
        
        for index, row in df.iterrows():
            question_text = row[question_col]  # Get the question from the DataFrame
            answer_text = row[answer_col]  # Get the reference answer
            
            print(f"Retrieving chunk for query: {question_text}")
            
            # Step 1: Retrieve the most relevant chunk from ChromaDB
            results = embed_and_query(question_text, model_norm, collections[law], top_k=1)
            if results and 'documents' in results and results['documents']:
                retrieved_chunk = results['documents'][0]  # Get the top retrieved result
                
                print(f"Summarizing retrieved chunk for query: {question_text}")
                
                # Step 2: Summarize the retrieved chunk
                summary = summarize_text_huggingface_with_retry(retrieved_chunk[0])  # Summarize the top chunk
                
                if summary:
                    print(f"Original answer: {answer_text}")
                    print(f"Retrieved chunk: {retrieved_chunk[0]}")
                    print(f"Generated summary: {summary}\n")
                    
                    # Step 3: Compare the summary with the original answer using cosine and semantic similarity
                    answer_embedding_cosine = model_cosine.encode([answer_text])[0]  # Embed with model_cosine
                    summary_embedding_cosine = model_cosine.encode([summary])[0]     # Embed with model_cosine
                    cosine_sim = calculate_cosine_similarity(answer_embedding_cosine, summary_embedding_cosine)

                    # Use model_semantic for semantic similarity
                    semantic_sim = calculate_semantic_similarity(answer_text, summary, model_semantic)
                    
                    # Store similarities
                    similarities[law]['cosine'].append(cosine_sim)
                    similarities[law]['semantic'].append(semantic_sim)
                    
                    print(f"Cosine similarity between summary and original answer: {cosine_sim:.4f}")
                    print(f"Semantic similarity between summary and original answer: {semantic_sim:.4f}")
                else:
                    print(f"Failed to generate a summary for query: {question_text}\n")
            else:
                print(f"No valid results found for query: {question_text} in {law.upper()} law")

    return similarities

# Function to calculate and print averages of cosine and semantic similarities
def calculate_and_print_averages(similarities):
    print("\nCalculated Averages:")
    for law, similarity_data in similarities.items():
        # Calculate average cosine similarity
        cosine_values = similarity_data['cosine']
        if cosine_values:
            avg_cosine = sum(cosine_values) / len(cosine_values)
        else:
            avg_cosine = None

        # Calculate average semantic similarity
        semantic_values = similarity_data['semantic']
        if semantic_values:
            avg_semantic = sum(semantic_values) / len(semantic_values)
        else:
            avg_semantic = None

        # Print the results
        if avg_cosine is not None and avg_semantic is not None:
            print(f"{law.upper()} Average Cosine Similarity: {avg_cosine:.4f}")
            print(f"{law.upper()} Average Semantic Similarity: {avg_semantic:.4f}")
        else:
            print(f"No valid similarities found for {law.upper()}")

# Store embeddings and chunks in ChromaDB
def process_and_store_embeddings(file_path, collection_name, model_norm):
    chunks = load_and_process_html(file_path)  # Get the text chunks
    
    # Generate embeddings using the appropriate method
    embeddings = model_norm.encode(chunks)  # Use the correct method for embeddings

    chroma_client = chromadb.Client()  # Initialize ChromaDB client
    collection = setup_chroma_collection(chroma_client, collection_name)  # Setup the collection
    
    for i, embedding in enumerate(embeddings):
        collection.add(
            documents=[chunks[i]],
            ids=[f"id_{i}"],
            embeddings=[embedding]
        )
    return collection

# Process the documents and store embeddings in ChromaDB
gdpr_collection = process_and_store_embeddings('/kaggle/input/chroma-db-italian/Italian_gdpr.html', 'gdpr_collection', model_norm)
ai_act_collection = process_and_store_embeddings('/kaggle/input/chroma-db-italian/italian_AI_act.html', 'ai_act_collection', model_norm)
dma_collection = process_and_store_embeddings('/kaggle/input/chroma-db-italian/italian_dma.html', 'dma_collection', model_norm)
dsa_collection = process_and_store_embeddings('/kaggle/input//chroma-db-italian/italian_dsa.html', 'dsa_collection', model_norm)

# Now add collections to a dictionary
collections = {
    'gdpr': gdpr_collection,
    'ai_act': ai_act_collection,
    'dma': dma_collection,
    'dsa': dsa_collection
}

# Summarize and compare for all laws
similarities = summarize_and_compare_all_laws(qa_dataframes, model_norm, collections)

# Calculate and print the averages of cosine and semantic similarities
calculate_and_print_averages(similarities)


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Cosine similarity between summary and original answer: 0.8598
Semantic similarity between summary and original answer: 0.4383

Calculated Averages:
GDPR Average Cosine Similarity: 0.9183
GDPR Average Semantic Similarity: 0.4710
AI_ACT Average Cosine Similarity: 0.9016
AI_ACT Average Semantic Similarity: 0.4714
DMA Average Cosine Similarity: 0.8921
DMA Average Semantic Similarity: 0.3635
DSA Average Cosine Similarity: 0.8798
DSA Average Semantic Similarity: 0.4297


In [29]:
import time

# mporei sfaksimo to ebala mono gia to kato

In [34]:
import torch

def generate_bert_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the last hidden state
    hidden_states = outputs.last_hidden_state  # Shape: (1, num_tokens, embedding_size)

    # Use the CLS token or mean-pool across tokens to get a single vector representation
    # CLS token method (often the first token):
    cls_embedding = hidden_states[:, 0, :].squeeze().numpy()  # Shape: (embedding_size,)

    # Or, mean-pooling method:
    mean_embedding = hidden_states.mean(dim=1).squeeze().numpy()  # Shape: (embedding_size,)

    return mean_embedding  # Choose either cls_embedding or mean_embedding


# den ksero apo pou

In [35]:
from IPython.display import display
from IPython.utils.io import capture_output

# Suppress unwanted output while calculating similarities
with capture_output() as captured:
    # Initialize dictionaries to store similarities for each law
    similarities = {
        'gdpr': {'cosine': [],},
        'ai_act': {'cosine': [],},
        'dma': {'cosine': [],},
        'dsa': {'cosine': [],}
    }

    for law, info in laws_info.items():
        for qa in info['questions_answers']:
            query = qa['question']
            reference_answer = qa['answer']

            # Embed and query
            results = embed_and_query(query, model_norm, collections[law], top_k=1)

            if results and 'documents' in results and results['documents']:
                retrieved_text = results['documents'][0][0]

                # Generate embeddings using BERT for cosine similarity
                retrieved_embedding = generate_bert_embedding(retrieved_text, tokenizer, model)
                reference_embedding = generate_bert_embedding(reference_answer, tokenizer, model)

                # Calculate similarities
                cosine_sim = calculate_cosine_similarity(reference_embedding, retrieved_embedding)

                # Store in the appropriate law's list
                similarities[law]['cosine'].append(cosine_sim)

# Calculate and print averages for each law
print("Calculated Averages:")
for law in similarities:
    avg_cosine = sum(similarities[law]['cosine']) / len(similarities[law]['cosine'])
    
    print(f"{law.upper()} Average Cosine Similarity: {avg_cosine:.4f}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Calculated Averages:
GDPR Average Cosine Similarity: 0.9616
AI_ACT Average Cosine Similarity: 0.9628
DMA Average Cosine Similarity: 0.9356
DSA Average Cosine Similarity: 0.9489


# based on the summaries cosine and semantic


# query latency i guess

In [31]:
import time

# Function to measure query latency and calculate averages
def measure_and_average_query_latency(laws_info, model_norm, collections, top_k=1):
    latency_results = {
        'gdpr': [],
        'ai_act': [],
        'dma': [],
        'dsa': []
    }
    all_latencies = []

    for law, info in laws_info.items():
        print(f"\nMeasuring query latency for {law.upper()} collection:")
        
        for qa in info['questions_answers']:
            query = qa['question']

            # Record start time
            start_time = time.time()

            # Embed and query
            results = embed_and_query(query, model_norm, collections[law], top_k)

            # Record end time
            end_time = time.time()

            # Calculate latency
            latency = end_time - start_time
            latency_results[law].append(latency)
            all_latencies.append(latency)

            print(f"Query: {query}")
            print(f"Latency: {latency:.4f} seconds")
            print("----\n")
    
    # Calculate and print average latency for each law
    for law in latency_results:
        if latency_results[law]:  # Check if the list is not empty
            avg_latency = sum(latency_results[law]) / len(latency_results[law])
            print(f"{law.upper()} Average Query Latency: {avg_latency:.4f} seconds")
        else:
            print(f"{law.upper()} has no recorded latencies.")

    # Calculate and print the overall average latency across all laws
    if all_latencies:
        overall_avg_latency = sum(all_latencies) / len(all_latencies)
        print(f"\nOverall Average Query Latency: {overall_avg_latency:.4f} seconds")
    else:
        print("No latencies recorded across all laws.")

# Run the latency measurement and averaging function
measure_and_average_query_latency(laws_info, model_norm, collections, top_k=1)



Measuring query latency for GDPR collection:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Qual è il diritto fondamentale relativo al trattamento dei dati personali secondo la Carta dei diritti fondamentali dell'Unione Europea?
Latency: 0.0495 seconds
----



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Quali sono i principi fondamentali del trattamento dei dati personali ai sensi del GDPR?
Latency: 0.0464 seconds
----



...

Query: Come devono le piattaforme online garantire la trasparenza nella moderazione dei contenuti secondo il DSA?
Latency: 0.0446 seconds
----



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Quali sono gli obblighi di notifica per le piattaforme che rimuovono o disabilitano l'accesso ai contenuti secondo il DSA?
Latency: 0.0452 seconds
----



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Quali sono le implicazioni per le piattaforme che non rispettano il DSA?
Latency: 0.0610 seconds
----

GDPR Average Query Latency: 0.0485 seconds
AI_ACT Average Query Latency: 0.0498 seconds
DMA Average Query Latency: 0.0472 seconds
DSA Average Query Latency: 0.0474 seconds

Overall Average Query Latency: 0.0482 seconds
